In [13]:
#테스트 진행 위한 준비
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import pandas as pd
import numpy as np
import itertools
import string
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

#우리가 사용할 모델
model = SentenceTransformer('all-mpnet-base-v2')


#문서 전처리 함수
def preprocess(doc):
    doc_list = []
    for i in doc:
        #구두점 제거
        doc1 = "".join([i for i in i if i not in string.punctuation]).strip()

        #숫자 제거
        doc2 = "".join([i for i in i if not i.isdigit()])

        #월 제거
        month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
                'september', 'october', 'november', 'december', 'jan', 'feb', 'mar', 'apr',
                 'may', 'jun','jul', 'aug', 'sep', 'oct', 'nov', 'dec']   
        
        doc3 = " ".join([i for i in doc2.split() if i not in month])
        
        #자체 stopword - 나라별로 생성한 stopword 사용
        stopword = ["australian", "australia" ,"duty", "abfgovauimportingexportingand", "manufacturingimportinghowtoimportdisposingunenteredabandonedgoods", 
           "declare", "consigment", "sorted", "license","goods", "products", "quota", "ii", "russia", "httpswwwabfgovauimporting", "customs",
                   "indexation", "working", "available", "subheadings", "cpi", "wwwabfgovau", "tariff", "office", "rates", "spirits", "rules", "blue",
                   "manufacturingtariffclassificationharmonizedsystemchanges"]

        doc4 = " ".join([i for i in doc3.split() if i not in stopword])
        doc_list.append(doc4)
    
    return doc_list

#문서 바이그램 단위로 나누는 함수 + nltk 제공하는 불용어 제거
def bigram(doc_list):
    n_gram_range = (2, 2)
    stop_words = "english"

    candidate_list = []
    for doc in doc_list:
        count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
        candidate = count.get_feature_names_out()
        candidate_list.append(candidate)
    
    return candidate_list

#전처리 문서와 바이그램 embedding 후 유사도 높은 30개 키워드 추출 함수
def bigram_embedding(model, doc_list, candidate_list):
    bigram_keywords = []
    top_n = 30

    for i in range(len(doc_list)):
        doc_embeddings = model.encode([doc_list[i]])
        candidate_embeddings = model.encode(candidate_list[i])
        distances = cosine_similarity(doc_embeddings, candidate_embeddings)
        bigram_keywords.append([candidate_list[i][index] for index in distances.argsort()[0][-top_n:]])
        
    return bigram_keywords


#추출 키워드 embedding 함수
def keyword_embedding(model, bigram_keywords):
    bigram_embeddings = []
    for i in range(len(doc_list)):
        bigram_embedding = []
        for keyword in bigram_keywords[i]:
            bigram_embedding.append(model.encode(keyword))
        bigram_embeddings.append(bigram_embedding)
    
    return bigram_embeddings

#선정 키워드(단어)와 embedding 값 불러오기
def get_keyword():
    df_keyword = pd.read_csv("호주_MTI_키워드.csv", index_col = False)
    keyword = list(df_keyword["번역 품목"])
    keyword_embeddings = []

    for ele in keyword:
        keyword_embeddings.append(model.encode(ele))
    
    return keyword, keyword_embeddings


#추출 키워드와 선정 키워드 유사도 비교 - 유사도는 나라별로 조정
def similarity_test(bigram_embeddings, keyword_embeddings, bigram_keywords, keyword):
    bigram_result = []
    keyword_result = []
    cosine_result = []
    for index, bigram in enumerate(bigram_embeddings):

        b_result = []
        k_result = []
        c_result = []

        for i in range(len(bigram)): 
            for j in range(len(keyword_embeddings)): 
                distances = cosine_similarity([bigram[i]],[keyword_embeddings[j]]) #유사도 비교
                if distances[0][0] > 0.5:
                    b_result.append(bigram_keywords[index][i])
                    k_result.append(keyword[j])
                    c_result.append(str(round(float(distances),3)))

        bigram_result.append(b_result)
        keyword_result.append(k_result)
        cosine_result.append(c_result)
        
    return bigram_result, keyword_result, cosine_result

#유사도 높은 순으로 df 만들기, 키워드는 최대 5개만 보여줄 예정
def make_df(bigram_result, keyword_result, cosine_result):
    df_final = pd.DataFrame()

    bigram_list = []
    keyword_list = []
    distance = []

    for i in range(len(df_add)):
        B, K, D = [], [], []
        b_result,k_result, d_result = "", "", ""
        df_check = pd.DataFrame()
        df_check['bigram'] = pd.Series(bigram_result[i])
        df_check['keyword'] = pd.Series(keyword_result[i])
        df_check['distance'] = pd.Series(cosine_result[i])

        df_check = df_check.sort_values(by="distance", ascending=False)
        
        #키워드가 5개보다 많으면 상위 5개만
        if len(df_check['bigram']) > 5:
            B = df_check['bigram'].tolist()[:5]
            K = df_check['keyword'].tolist()[:5]
            D = df_check['distance'].tolist()[:5]
        #키워드가 5개보다 적으면 전체 보여주기
        else:
            B = df_check['bigram'].tolist()
            K = df_check['keyword'].tolist()
            D = df_check['distance'].tolist()


        
        for b in B:
            b_result = b_result + "/" + b
        for k in K:
            k_result = k_result + "/" + k
        for d in D:
            d_result = d_result + "/" + d


        bigram_list.append(b_result[1:])
        keyword_list.append(k_result[1:])
        distance.append(d_result[1:])

        del df_check

    df_final['bigram'] = bigram_list
    df_final['keyword'] = keyword_list
    df_final['distance'] = distance

    return df_final


In [2]:
#키워드 뽑은 파일 df로 불러오기
df = pd.read_csv("호주_세관고시_번역_hs(결측치 제거).csv")

#제목 + 본문 = 문서
df_title = list(df['title'])
df_text = list(df['text'])
df_add = []

#제목 + 본문을 하나의 문서로 고려하기 때문에 합치는 작업
for i in range(len(df)):
    df_add.append(str(df_title[i]).lower()+""+str(df_text[i]))

In [3]:
df_add = df_add[:20]

In [4]:
#테스트 코드 - 자체 stopword 생성 위해 테스트 진행하는 경우 아래 과정 따로 진행하는게 더 빠름

#문서 전처리 진행
doc_list = preprocess(df_add)

In [5]:
#문서에서 바이그램 추출
candidate_list = bigram(doc_list)

In [8]:
#전처리 문서와 바이그램 유사도 비교 후 top 30 바이그램 선정
bigram_keywords = bigram_embedding(model, doc_list, candidate_list) 

In [10]:
#top 30 바이그램 embedding 진행
bigram_embeddings = keyword_embedding(model, bigram_keywords) 

In [15]:
#선정 키워드(단어) 불러오기
keyword, keyword_embeddings = get_keyword()

In [21]:
#유사도 검사
bigram_result, keyword_result, cosine_result = similarity_test(bigram_embeddings, keyword_embeddings, bigram_keywords, keyword)

In [22]:
#유사도 큰 순으로 정렬하는 df 생성

#합치려고 하는 파일 df로 불러오기
df1 = pd.read_csv("호주_세관고시_번역_hs(결측치 제거).csv", index_col = False)

#키워드 df 생성
df2 = make_df(bigram_result, keyword_result, cosine_result)

#합친 df 생성
df3 = pd.concat([df1,df2],axis=1)

C:\Users\water\AppData\Local\Temp\ipykernel_13552\2643017289.py:134: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_check['bigram'] = pd.Series(bigram_result[i])
C:\Users\water\AppData\Local\Temp\ipykernel_13552\2643017289.py:135: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_check['keyword'] = pd.Series(keyword_result[i])
C:\Users\water\AppData\Local\Temp\ipykernel_13552\2643017289.py:136: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_check['distance'] = pd.Series(cosine_result[i])
C:\Users\water\AppData\Local\Temp\ipykernel_13552\2643017289.py:134: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float

In [23]:
df3

,date,link,title,text,title_ko,hscode,bigram,keyword,distance
0,2022-07-28,https://www.abf.gov.au/help-and-support-subsit...,Indexation of customs duty rates on excise-equ...,australian customs notice no. 2022/35 indexat...,2022년 8월 1일 소비재 등가물품에 대한 관세율 지표,2202.99/3825.69/2903.41/3824.89/2710.99/2701.1...,,,
1,2022-07-22,https://www.abf.gov.au/help-and-support-subsit...,Removal of customs duty on certain electric ve...,australian customs notice no. 2022/34 removal...,특정 전기 차량에 대한 관세 철폐,8701.92/8701.29/8701.94/8703.70/8704.31/8701.9...,certain electric/vehicles electric/vehicles el...,an electric instrument/an automobile internal ...,0.607/0.572/0.553/0.541/0.527
2,2022-07-21,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,australian customs notice no. 2022/33 applica...,관세 브로커 면허 신청서,NaN,,,
3,2022-07-12,https://www.abf.gov.au/help-and-support-subsit...,Temporary duty reduction for goods from Ukraine,australian customs notice no. 2022/32 tempora...,우크라이나 물품에 대한 임시 관세 감면,NaN,,,
4,2022-07-11,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,australian customs notice no. 2022/31 applica...,관세 브로커 면허 신청서,NaN,cargo business,ship,0.546
...,...,...,...,...,...,...,...,...,...
1614,2018-12-10,https://www.abf.gov.au/help-and-support-subsit...,Preparation of submanifests for export cargo c...,australian customs notice 1996 09 preparation...,수출화물통합부속서 작성개시일 연기,NaN,NaN,NaN,NaN
1615,2018-12-10,https://www.abf.gov.au/help-and-support-subsit...,Section 135 of the Copyright Act 1968,australian customs notice 1996 07 section 135...,1968년 저작권법 제135조,NaN,NaN,NaN,NaN
1616,2018-12-10,https://www.abf.gov.au/help-and-support-subsit...,Sea cargo automation: electronic transhipment ...,australian customs notice 1996 06 sea cargo a...,해상 화물 자동화: 환적 화물에 대한 전자 환적 해제,NaN,NaN,NaN,NaN
1617,2018-12-10,https://www.abf.gov.au/help-and-support-subsit...,Indexation of certain excise and customs tarif...,australian customs notice 1996 05 29 indexati...,특정 소비세 및 관세율 및 디젤 연료 리베이트의 지수화,2208.20/2208.30/2208.40/2206.00/2402.20/2208.9...,NaN,NaN,NaN


In [24]:
#합친 df 파일로 만들기 - 필요한 제목 입력
df3.to_csv('호주_세관고시_번역_mti_키워드.csv', mode='w', encoding='utf-8-sig', index = False)